In [62]:
import functools
raw = ''

In [63]:
with open('./example-07.txt','r') as file:
    raw = file.read()


In [64]:
data1 = [x.split(' ') for x in raw.split('\n')]
data1

[['$', 'cd', '/'],
 ['$', 'ls'],
 ['dir', 'a'],
 ['14848514', 'b.txt'],
 ['8504156', 'c.dat'],
 ['dir', 'd'],
 ['$', 'cd', 'a'],
 ['$', 'ls'],
 ['dir', 'e'],
 ['29116', 'f'],
 ['2557', 'g'],
 ['62596', 'h.lst'],
 ['$', 'cd', 'e'],
 ['$', 'ls'],
 ['584', 'i'],
 ['$', 'cd', '..'],
 ['$', 'cd', '..'],
 ['$', 'cd', 'd'],
 ['$', 'ls'],
 ['4060174', 'j'],
 ['8033020', 'd.log'],
 ['5626152', 'd.ext'],
 ['7214296', 'k']]

In [65]:
class Dir:
    def __init__(self, name) -> None:
        self.name = name
        self.dirs = {}
        self.files = {}

    class File:
        def __init__(self,name,size) -> None:
            self.name = name
            self.size = size

        def __str__(self):
            return f'{self.name} : {self.size}'
            
        def get_size(self) -> int:
            return self.size

        def get_max_size(self) -> int:
            return self.size if self.size <= 100000 else 0
        

    def __str__(self) -> str:
        return f'{self.name} : {self.dirs}'

    def get_size(self) -> int:
        total = 0

        for _,dir in self.dirs.items():
            total += dir.get_size()

        for _,file in self.files.items():
            total += file.get_size()

        return total   

    def get_max_size(self) -> int:
        total = 0
        local = 0

        for _,dir in self.dirs.items():
            local += dir.get_size()

        for _,file in self.files.items():
            local += file.get_size()

        for _,dir in self.dirs.items():
            total += dir.get_max_size()

        if local <= 100000:
            total += local
        return total

    def get_dir(self):
        dirs = []
        dirs.append(self.get_size())

        for _,dir in self.dirs.items():
            dirs = dirs + dir.get_dir()
        
        return dirs

    def add_file(self, name, size) -> None:
        self.files[name] = self.File(name,size)


In [66]:
def scan(dir,name, DIR):
    global idx
    global line
    global size
    
    total = 0
    line= data1[idx]
    if(line[0] == '$'):
        if (line[1] == 'ls'):
            while True:
                idx += 1
                if idx >= max: break
                line = data1[idx]

                if line[0] == '$': break
                
                if (line[0].isnumeric()):
                    dir[line[1]] = int(line[0])
                    
                    DIR.add_file(line[1] , int(line[0]))

                    if name == 'zrfmzrl': print(dir) 
                    total += int(line[0])
                else:
                    DIR.dirs[line[1]] = Dir(line[1])
                    dir[line[1]] = dict()
                

    while idx < max:
        line = data1[idx]
        # if idx >= 15: print(line)
        idx += 1
        if line[1] == 'cd':
            if line[2] == '..':
                size[name] = total
                return total
            DIR.dirs[line[2]] = Dir(line[2])
            total = total + scan(dir[line[2]], line[2], DIR.dirs[line[2]])
            size[name] = total
    size[name] = total
    return total


In [67]:
dirs = {}
dirs['/'] = {}
size = {}
idx = 1
line = []
max = len(data1)
root = Dir('/')
scan(dirs,'/', root)

48381165

In [68]:
functools.reduce(lambda a,b: a+b,  list(filter(lambda x: x <= 100000,list(size.values()))))

95437

In [69]:
max_size = 0
root.get_max_size()

95437

# Parte 2

In [70]:
diff = root.get_size() - 40000000
diff

8381165

In [71]:
result = root.get_dir()
result.sort()
result

[584, 94853, 24933642, 48381165]

In [72]:
next((item for item in result if item > diff),None)

24933642